# Crawling class tests

## Scholar crawling tests

In [1]:
import django
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "byteme.settings.base")
django.setup()

In [2]:
from crawler.models import Crawler
from accounts.models import Speaker, UserProfile
from events.tag import Tag
from events.models import Event
from time import sleep

In [3]:
my_crawler = Crawler()

In [4]:
#This failed
univ_list = ['Kaist', 'Korea Advanced Institute of Science and Technology', 'Stanford',
       'Cambridge', 'MIT', 'Yale', 'Georgia Tech', 'Harvard', 'ETH Zurich', 'EPFL', 'Oxford',
       'Imperial College London', 'NUS', 'NTU', 'Princeton', 'Cornell', 'Tshinghua']
univ_scholar_ids = []
for count, univ in enumerate(univ_list):
    try:
        ids = my_crawler.crawl_univ_scholar_ids(univ, 20)
        univ_scholar_ids.append(ids)
        print(count, univ, len(ids))
    except Exception:
        print('Exception Occured')
    sleep(10)

Exception Occured


KeyboardInterrupt: 

In [ ]:
s1 = Speaker(name='Min H Kim', speakerEmail='1@kaist.ac.kr')
s2 = Speaker(name='Daniel Suk Jeon', speakerEmail='2@kaist.ac.kr')
s3 = Speaker(name='Osman', univ='Stanford', speakerEmail='3@kaist.ac.kr')
s1.save()
s2.save()
s3.save()
print(Speaker.objects.all())

In [ ]:
s1 = Speaker.objects.all()[0]
s2 = Speaker.objects.all()[1]
s3 = Speaker.objects.all()[2]

In [ ]:
my_crawler.scholar_crawl_request(s1)
my_crawler.scholar_crawl_request(s2)
my_crawler.scholar_crawl_request(s3)

In [ ]:
print(Tag.objects.all())
for speaker in Speaker.objects.all():
    print(speaker.tags.all())

## Indiv Score testing

In [ ]:
print('UserProfiles: ', UserProfile.objects.all())
print('Speakers: ', Speaker.objects.all())
print('Tags:', Tag.objects.all())
print('Events:', Event.objects.all())

In [ ]:
event = Event.objects.all()[0]
user = UserProfile.objects.all()[0]

In [ ]:
score = event.generateRankingScore(user)
print(score)

In [ ]:
event.speaker.citations

## Pytrends Test

In [ ]:
import pytrends
from pytrends.request import TrendReq

pytrend = TrendReq(hl='en-US', tz=360)
#pytrend = TrendReq(hl='en-US', tz=360, proxies = {'https': 'https://34.203.233.13:80'})

In [ ]:
#kw_list = ["Blockchain"]
kw_list = ['generative adverserial network', 'neural machine translation', 'neural turing machine']
pytrend.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='', gprop='')

In [ ]:
interest_over_time_df = pytrend.interest_over_time()
print(interest_over_time_df)

In [ ]:
def parse_scholar_id(in_str):
    idx1 = in_str.find('user=') + 5
    idx2 = in_str.find('&') #Check this!!
    substr = in_str[idx1:idx2]
    return substr

def create_link(scholar_id):
    return "http://scholar.google.com/citations?user=" + scholar_id + "&hl=en"

In [ ]:
query = "Dieter Fox google scholar"

matches = set()
for j in search(query, stop=5): 
    if "scholar.google" in j and len(j) < 100:
        scholar_id = parse_scholar_id(j)
        matches.add(scholar_id)
        
print('Number of links: ', len(matches))

In [ ]:
scholar_id = matches.pop()
link = create_link(scholar_id)
print(link)
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())